# 문장 쌍 분류

* 문장(또는 문서)가 2개 주어졌을 때 해당 문장 사이의 관계가 어떤 범주일지 분류하는 과제

ex) 자연어 추론 -> 두 문장의 관계가 참인지, 거짓인지, 중립 또는 판단 불가인지 가려내는 것
<br>
KLUE-NLI 데이터셋을 가지고 Sentence Pair Classification Model 구축

### NLI(Natural Language Inference

자연어 추론은 2개의 문장(또는 문서)이 참(entailment), 거짓(contradiction), 중립 또는 판단 불가(neutral)인지 가려내는 것
<br>
> 나 출근했어 + 난 백수야 -> 거짓 <br>
나 출근했어 + 난 개발자다 -> 중립

데이터셋 : NLI 데이터셋 -> Premise(전제)에 대한 가설이 참인지, 거짓인지, 중립인지 정보가 Label(gole_label)로 주어져 있다.

<NLI 데이터셋 예시>
|  | - | - | - |
|---|---|---|---|
| **전체** | 100분간 잘껄 그래도 소닉붐땜에 2점준다 | 100분간 잘껄 그래도 소닉붐땜에 2점준다 | 101빌딩 근처에 나름 즐길거리가 많습니다. |
| **가설** | 100분간 잤다. | 소닉붐이 정말 멋있었다. | 101빌딩 부근에서는 여러가지를 즐길 수 있습니다. |
| **레이블** | contradiction | neutral | entailment |

<br>
NLI 과제 수행 모델 -> 전제와 가설 2개 문장을 입력으로 하고, 두 문장의 관계가 어떤 범주일지 확률을 출력

* 100분간 잘껄 그래도 소닉붐땜에 2점준다(전제) + 100분간 잤다.(가설) → [0.02, 0.97, 0.01](참, 거짓, 중립일 확률) → contradiction(후처리 결과)
* 100분간 잘껄 그래도 소닉붐땜에 2점준다(전제) + 소닉붐이 정말 멋있었다.(가설) → [0.01, 0.01, 0.98](참, 거짓, 중립일 확률) → neutral(후처리 결과)

<br>

### 모델 구조

전제와 가설 두 문장을 각각 토큰화한 뒤 <b>[CLS]+전제+[SEP]+가설+[SEP]</b>형태로 이어 붙인다.<br>
CLS - 문장 시작, SEP - 전제와 가설을 구분해주는 스페셜 토큰<br>
이를 BERT 모델에 입력하고 문장 수준의 벡터(Poller_output)를 뽑는다.<br<
이 벡터에는 전제와 가설의 의미가 응축되어 있다.<br>
작은 추가 모듈을 덧붙여 모델 전체의 출력이 [전제에 대해 가설이 참일 확률, 전제에 대해 가설이 거짓일 확률, 전제에 대한 가설이 중립일 확률] 형태가 ㅗ되게한다.

### 태스크 모듈
<문장 쌍 분류 태스크 모듈>
<img src="task_module.png">
출처 : ratsgo's NLPBOOK
<b>Pooler_output</b> 벡터 뒤에 붙는 추가 모듈의 구조는 다음과 같다.<br>
우선 <b>Pooler_output</b>을 분류해야할 범주 수만큼의 차원을 갖는 벡터로 변환.<br>
만약에 <b>Pooler_output</b> 벡터가 768차원이고 분류 대상 범주 수가 3개(참,거짓,중립)라면 가중치 행렬 크기는 $768X3$이 된다.<br>
여기에 Softmax 함수를 취하면 모델의 최종 출력이 된다. 모델의 최종출려과 정답 레이블을 비교해 모델 출력이 정답 레이블과 같아지도록 BERT 레이어를 포함한 모델 전체를 업데이트
<br>
문장 쌍 분류 태스크 모듈은 4장에서 다른 문서 분류 태스크 모듈과 거의 유사한 모습<br>
문서 분류 과제를 3개 범주(긍정, 부정, 중립)를 분류하는 태스크로 상정한다면 두 모듈 구조는 똑같다. <br>다만 차이는 태스크 모듈의 입력(pooler_output)이 된다. <br>즉, pooler_output에 문장 1개의 의미가 응축되어 있다면 문서 분류, 2개의 의미가 내포해 있다면 문장 쌍 분류 과제가 된다.


$ python my_flask_script.py

my_flask_script.py -> index.html -> my_flask_script.py

evaluate.py -> 평가

 $ ngrok http 5001

In [3]:
import torch
import random
import numpy as np
import os
from transformers import BertTokenizer, BertConfig, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler, SequentialSampler
import torch.nn.functional as F

# 랜덤 시드 고정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(7)

# 학습 하이퍼파라미터 설정
class TrainArgs:
    def __init__(self):
        self.pretrained_model_name = "beomi/kcbert-base"
        self.batch_size = 32 if torch.cuda.is_available() else 4
        self.learning_rate = 5e-5
        self.max_seq_length = 64
        self.epochs = 5
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.output_dir = "./bert_nli_model"

args = TrainArgs()

# 토크나이저 준비
tokenizer = BertTokenizer.from_pretrained(args.pretrained_model_name, do_lower_case=False)

# 데이터셋 로드 및 전처리
dataset = load_dataset("klue", "nli")

def preprocess_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True, padding="max_length", max_length=args.max_seq_length)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# 데이터셋 포맷 변경 (PyTorch에서 사용 가능하도록)
train_dataset = tokenized_datasets["train"]
val_dataset = tokenized_datasets["validation"]

# 모델 초기화
model_config = BertConfig.from_pretrained(args.pretrained_model_name, num_labels=3)
model = BertForSequenceClassification.from_pretrained(args.pretrained_model_name, config=model_config)
model.to(args.device)

# 학습 설정
training_args = TrainingArguments(
    output_dir=args.output_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=args.learning_rate,
    per_device_train_batch_size=args.batch_size,
    per_device_eval_batch_size=args.batch_size,
    num_train_epochs=args.epochs,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=200,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# 모델 학습
trainer.train()

# 학습된 모델 저장
trainer.save_model(args.output_dir)

# 모델 로드
model = BertForSequenceClassification.from_pretrained(args.output_dir)
model.to(args.device)
model.eval()




train-00000-of-00001.parquet:   0%|          | 0.00/1.83M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/224k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24998 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/24998 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Epoch,Training Loss,Validation Loss
1,0.973100,1.010658
2,0.895900,1.043298
3,0.835200,1.119043
4,0.705700,1.184783
5,0.668500,1.252972


In [8]:
# 모델과 토크나이저 저장
model.save_pretrained("./bert_nli_model")
tokenizer.save_pretrained("./bert_nli_model")

('./bert_nli_model/tokenizer_config.json',
 './bert_nli_model/special_tokens_map.json',
 './bert_nli_model/vocab.txt',
 './bert_nli_model/added_tokens.json')

# 다시 불러올 때
from transformers import BertTokenizer, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("./bert_nli_model")
tokenizer = BertTokenizer.from_pretrained("./bert_nli_model")
model.eval()

In [14]:
def inference_fn(premise, hypothesis):
    inputs = tokenizer([(premise, hypothesis)], max_length=args.max_seq_length, padding="max_length", truncation=True, return_tensors="pt")
    inputs = {key: value.to(args.device) for key, value in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        prob = F.softmax(outputs.logits, dim=1)
        predicted_index = torch.argmax(prob, dim=1).item()

        # ✅ KLUE NLI의 실제 라벨 순서로 수정
        LABEL_MAPPING = {
            1: "참 (entailment)",       # index 1이 "참"
            0: "거짓 (contradiction)",  # index 0이 "거짓"
            2: "중립 (neutral)"         # index 2가 "중립"
        }
    
        pred = LABEL_MAPPING[predicted_index]

    return {
        'premise': premise,
        'hypothesis': hypothesis,
        'prediction': pred,
        'entailment_data': f"참 {round(prob[0][1].item(), 2)}",  # index 1이 entailment
        'contradiction_data': f"거짓 {round(prob[0][0].item(), 2)}",  # index 0이 contradiction
        'neutral_data': f"중립 {round(prob[0][2].item(), 2)}",  # index 2가 neutral
        'entailment_width': f"{prob[0][1].item() * 100}%",
        'contradiction_width': f"{prob[0][0].item() * 100}%",
        'neutral_width': f"{prob[0][2].item() * 100}%",
    }

In [11]:
# Flask 웹 서비스
from flask import Flask, request, jsonify

app = Flask(__name__)

# 루트 경로 ('/') 추가
@app.route('/')
def home():
    return "Flask NLI Model is running! Use /predict endpoint for inference."

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    premise = data.get("premise")
    hypothesis = data.get("hypothesis")

    if not premise or not hypothesis:
        return jsonify({"error": "premise and hypothesis are required"}), 400

    result = inference_fn(premise, hypothesis)
    return jsonify(result)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001)  # 포트 5001에서 실행

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://172.30.1.1:5001
Press CTRL+C to quit
127.0.0.1 - - [14/Mar/2025 00:03:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2025 00:03:47] "GET /predict HTTP/1.1" 405 -
127.0.0.1 - - [14/Mar/2025 00:05:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2025 00:05:47] "POST /predict HTTP/1.1" 200 -


In [15]:
print(inference_fn("나는 남자야", "나는 여자야"))

{'premise': '나는 남자야', 'hypothesis': '나는 여자야', 'prediction': '거짓 (contradiction)', 'entailment_data': '참 0.03', 'contradiction_data': '거짓 0.91', 'neutral_data': '중립 0.06', 'entailment_width': '3.1154492869973183%', 'contradiction_width': '90.71720242500305%', 'neutral_width': '6.167354062199593%'}
